##### Name: Stuti Upadhyay
##### Campus ID: XT81177
##### Instructor: Chalachew Jemberie

## Homework-Logistic Regression


### Dataset

In this homework, we will use the California Housing Prices data from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).


We'll keep working with the `'median_house_value'` variable, and we'll transform it to a classification task. 

### Features

For the rest of the homework, you'll need to use only these columns:

* `'latitude'`,
* `'longitude'`,
* `'housing_median_age'`,
* `'total_rooms'`,
* `'total_bedrooms'`,
* `'population'`,
* `'households'`,
* `'median_income'`,
* `'median_house_value'`,
* `'ocean_proximity'`,

### Data preparation

* Select only the features from above and fill in the missing values with median.
* Create a new column `rooms_per_household` by dividing the column `total_rooms` by the column `households` from dataframe. 
* Create a new column `bedrooms_per_room` by dividing the column `total_bedrooms` by the column `total_rooms` from dataframe. 
* Create a new column `population_per_household` by dividing the column `population` by the column `households` from dataframe. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.feature_selection import mutual_info_regression

# Load your dataset
data = pd.read_csv("housing.csv")

In [2]:
# Select only the specified features
selected_features = ['latitude', 'longitude', 'housing_median_age', 'total_rooms', 
                     'total_bedrooms', 'population', 'households', 'median_income', 
                     'median_house_value', 'ocean_proximity']
housing_data = data[selected_features]

In [3]:
# Fill missing values for numerical columns with median
numerical_cols = housing_data.select_dtypes(include=['number']).columns
housing_data[numerical_cols] = housing_data[numerical_cols].fillna(housing_data[numerical_cols].median())

In [4]:
# Create new columns
housing_data['rooms_per_household'] = housing_data['total_rooms'] / housing_data['households']
housing_data['bedrooms_per_room'] = housing_data['total_bedrooms'] / housing_data['total_rooms']
housing_data['population_per_household'] = housing_data['population'] / housing_data['households']

In [5]:
# Display the updated dataframe
print(housing_data.head())

   latitude  longitude  housing_median_age  total_rooms  total_bedrooms  \
0     37.88    -122.23                41.0        880.0           129.0   
1     37.86    -122.22                21.0       7099.0          1106.0   
2     37.85    -122.24                52.0       1467.0           190.0   
3     37.85    -122.25                52.0       1274.0           235.0   
4     37.85    -122.25                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  \
0       322.0       126.0         8.3252            452600.0        NEAR BAY   
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY   
2       496.0       177.0         7.2574            352100.0        NEAR BAY   
3       558.0       219.0         5.6431            341300.0        NEAR BAY   
4       565.0       259.0         3.8462            342200.0        NEAR BAY   

   rooms_per_household  bedrooms_per_room  population_per_household 

### Question 1

What is the most frequent observation (mode) for the column `ocean_proximity`?

Options:
* `NEAR BAY`
* `<1H OCEAN`
* `INLAND`
* `NEAR OCEAN`

In [6]:
mode_ocean_proximity = housing_data['ocean_proximity'].mode()[0]
print("Mode of ocean_proximity:", mode_ocean_proximity)

Mode of ocean_proximity: <1H OCEAN


### Question 2

* Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your train dataset.
    - In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

Options:
* `total_bedrooms` and `households`
* `total_bedrooms` and `total_rooms`
* `population` and `households`
* `population_per_household` and `total_rooms`


### Make `median_house_value` binary

* We need to turn the `median_house_value` variable from numeric into binary.
* Let's create a variable `above_average` which is `1` if the `median_house_value` is above its mean value and `0` otherwise.

### Split the data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
* Make sure that the target value (`median_house_value`) is not in your dataframe.

In [7]:
# Select only numerical features
numerical_columns = housing_data.select_dtypes(include=['number']).columns
housing_data_numeric = housing_data[numerical_columns]

# Calculate the correlation matrix
correlation_matrix = housing_data_numeric.corr()
print(correlation_matrix)

                          latitude  longitude  housing_median_age  \
latitude                  1.000000  -0.924664            0.011173   
longitude                -0.924664   1.000000           -0.108197   
housing_median_age        0.011173  -0.108197            1.000000   
total_rooms              -0.036100   0.044568           -0.361262   
total_bedrooms           -0.066484   0.069120           -0.319026   
population               -0.108785   0.099773           -0.296244   
households               -0.071035   0.055310           -0.302916   
median_income            -0.079809  -0.015176           -0.119034   
median_house_value       -0.144160  -0.045967            0.105623   
rooms_per_household       0.106389  -0.027540           -0.153277   
bedrooms_per_room        -0.098619   0.081205            0.135622   
population_per_household  0.002366   0.002476            0.013191   

                          total_rooms  total_bedrooms  population  households  \
latitude             

In [8]:
# Split the data
X = housing_data.drop(columns=['median_house_value'])  # Features
y = (housing_data['median_house_value'] > housing_data['median_house_value'].mean()).astype(int)  # Target variable
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Question 3

* Calculate the *mutual information score* between `above_average` and `ocean_proximity` . Use the training set only.
* Round it to 2 decimals using `round(score, 2)`
* What is their *mutual information score*?


Options:
- 0.26
- 0
- 0.10
- 0.16

In [9]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the 'ocean_proximity' column
encoder = OneHotEncoder()
X_train_encoded = encoder.fit_transform(X_train[['ocean_proximity']])

In [10]:
# Compute mutual information score
mi_score = mutual_info_regression(X_train_encoded, y_train)

In [11]:
# Compute mutual information score
mi_scores_rounded = [round(score, 2) for score in mi_score]
print("Mutual information scores:", mi_scores_rounded)

Mutual information scores: [0.02, 0.1, 0.0, 0.01, 0.01]


### Question 4

* Now let's train a logistic regression
* Remember that we have one categorical variable `ocean_proximity` in the data. Include it using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

Options:
- 0.60
- 0.72
- 0.84
- 0.95

In [12]:
# One-hot encode categorical variable
X_train_encoded = pd.get_dummies(X_train, columns=['ocean_proximity'])
X_val_encoded = pd.get_dummies(X_val, columns=['ocean_proximity'])

In [13]:
# Train logistic regression model
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [14]:
# Calculate accuracy on validation set
val_predictions = model.predict(X_val_encoded)
accuracy = accuracy_score(y_val, val_predictions)
rounded_accuracy = round(accuracy, 2)
print("Accuracy on validation set:", rounded_accuracy)

Accuracy on validation set: 0.83


### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 
* Which of following feature has the smallest difference? 
   * `total_rooms`
   * `total_bedrooms` 
   * `population`
   * `households`

> **Note**: the difference doesn't have to be positive

In [15]:
# Train logistic regression model with all features
model_all_features = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model_all_features.fit(X_train_encoded, y_train)
accuracy_all_features = accuracy_score(y_val, model_all_features.predict(X_val_encoded))

In [16]:
# Initialize dictionary to store feature importance differences
feature_importance_diff = {}

In [17]:
# Exclude each feature and train a model without it
for feature in X_train_encoded.columns:
    X_train_subset = X_train_encoded.drop(columns=[feature])
    X_val_subset = X_val_encoded.drop(columns=[feature])
    
    model_subset = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model_subset.fit(X_train_subset, y_train)
    
    accuracy_subset = accuracy_score(y_val, model_subset.predict(X_val_subset))
    
    # Calculate difference in accuracy
    diff = accuracy_all_features - accuracy_subset
    feature_importance_diff[feature] = diff

In [18]:
# Find the feature with the smallest difference
smallest_diff_feature = min(feature_importance_diff, key=feature_importance_diff.get)
print("Feature with smallest difference:", smallest_diff_feature)

Feature with smallest difference: population_per_household


### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'median_house_value'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model (`model = Ridge(alpha=a, solver="sag", random_state=42)`) on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

Options:
- 0
- 0.01
- 0.1
- 1
- 10

In [19]:
# One-hot encode categorical variable
X_train_encoded = pd.get_dummies(X_train, columns=['ocean_proximity'])

In [20]:
X_val_encoded = pd.get_dummies(X_val, columns=['ocean_proximity'])

In [21]:
# Apply logarithmic transformation to 'median_house_value'
housing_data['median_house_value_log'] = housing_data['median_house_value'].apply(lambda x: np.log(x))

In [22]:
# Split the data
X = housing_data.drop(columns=['median_house_value', 'median_house_value_log'])  # Features
y = housing_data['median_house_value_log']  # Target variable
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [23]:
# Train Ridge regression models with different alpha values
alphas = [0, 0.01, 0.1, 1, 10]
best_alpha = None
best_rmse = float('inf')

for alpha in alphas:
    model = Ridge(alpha=alpha, solver="sag", random_state=42)
    model.fit(X_train_encoded, y_train)
    val_predictions = model.predict(X_val_encoded)
    rmse = mean_squared_error(y_val, val_predictions, squared=False)
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_alpha = alpha

/Users/stuti_up_02/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/stuti_up_02/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/stuti_up_02/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/stuti_up_02/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/stuti_up_02/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [24]:
rounded_rmse = round(best_rmse, 3)
print("Best RMSE on validation set:", rounded_rmse, "for alpha =", best_alpha)

Best RMSE on validation set: 0.531 for alpha = 0
